# High-level CNN Keras (TF) Example

*Modified by Jordan A Caraballo Vega (jordancaraballo)*

In [1]:
import os
import sys
import numpy as np
os.environ['KERAS_BACKEND'] = "tensorflow"

import warnings # make notebook more readable and nice
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=DeprecationWarning)
from tensorflow.python.util import deprecation
deprecation._PRINT_DEPRECATION_WARNINGS = False

import keras as K
import tensorflow
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, Dropout
from common.params import *
from common.utils import *

Using TensorFlow backend.


In [2]:
# Force one-gpu
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [3]:
# Performance Improvement
# 1. Make sure channels-first (not last)
K.backend.set_image_data_format('channels_first')

In [4]:
print("OS: ", sys.platform)
print("Python: ", sys.version)
print("Keras: ", K.__version__)
print("Numpy: ", np.__version__)
print("Tensorflow: ", tensorflow.__version__)
print(K.backend.backend())
print(K.backend.image_data_format())
print("GPU: ", get_gpu_name())
print(get_cuda_version())
print("CuDNN Version ", get_cudnn_version())

OS:  linux
Python:  3.6.7 | packaged by conda-forge | (default, Jul  2 2019, 02:18:42) 
[GCC 7.3.0]
Keras:  2.2.4
Numpy:  1.17.0
Tensorflow:  1.13.1
tensorflow
channels_first
GPU:  ['Tesla V100-SXM2-16GB', 'Tesla V100-SXM2-16GB', 'Tesla V100-SXM2-16GB', 'Tesla V100-SXM2-16GB']
CUDA Version 10.1.243
CuDNN Version  No CUDNN in this machine


In [5]:
def create_symbol(n_classes=N_CLASSES):
    model = Sequential()
    
    model.add(Conv2D(50, kernel_size=(3, 3), padding='same', activation='relu',
                     input_shape=(3, 32, 32)))
    model.add(Conv2D(50, kernel_size=(3, 3), padding='same', activation='relu'))    
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Dropout(0.25))
    
    model.add(Conv2D(100, kernel_size=(3, 3), padding='same', activation='relu'))
    model.add(Conv2D(100, kernel_size=(3, 3), padding='same', activation='relu'))    
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Dropout(0.25))
        
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(n_classes, activation='softmax'))
    return model

In [6]:
def init_model(m, lr=LR, momentum=MOMENTUM):
    m.compile(
        loss = "categorical_crossentropy",
        optimizer = K.optimizers.SGD(lr, momentum),
        metrics = ['accuracy'])
    return m

In [7]:
%%time
# Data into format for library
x_train, x_test, y_train, y_test = cifar_for_library(channel_first=True, one_hot=True)
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)
print(x_train.dtype, x_test.dtype, y_train.dtype, y_test.dtype)

Preparing train set...
Preparing test set...


/home/jacaraba/.conda/envs/ngaml/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:441: DeprecationWarning: The 'categorical_features' keyword is deprecated in version 0.20 and will be removed in 0.22. The passed value of 'all' is the default and can simply be removed.
  DeprecationWarning)


(50000, 3, 32, 32) (10000, 3, 32, 32) (50000, 10) (10000, 10)
float32 float32 int32 int32
CPU times: user 531 ms, sys: 738 ms, total: 1.27 s
Wall time: 3.4 s


In [8]:
%%time
# Load symbol
sym = create_symbol()

CPU times: user 1.05 s, sys: 1.51 s, total: 2.56 s
Wall time: 26.8 s


In [9]:
%%time
# Initialise model
model = init_model(sym)

CPU times: user 27 ms, sys: 3.27 ms, total: 30.3 ms
Wall time: 27.9 ms


In [10]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 50, 32, 32)        1400      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 50, 32, 32)        22550     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 50, 16, 16)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 50, 16, 16)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 100, 16, 16)       45100     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 100, 16, 16)       90100     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 100, 8, 8)         0         
__________

In [11]:
%%time
# Main training loop: 1m16s
EPOCHS=40
model.fit(x_train,
          y_train,
          batch_size=BATCHSIZE,
          epochs=EPOCHS,
          verbose=1)

Epoch 1/40
50000/50000 [==============================] - 27s 533us/step - loss: 2.0714 - acc: 0.2359
Epoch 2/40
50000/50000 [==============================] - 3s 65us/step - loss: 1.6919 - acc: 0.3905
Epoch 3/40
50000/50000 [==============================] - 3s 67us/step - loss: 1.4951 - acc: 0.4612
Epoch 4/40
50000/50000 [==============================] - 3s 66us/step - loss: 1.3482 - acc: 0.5151
Epoch 5/40
50000/50000 [==============================] - 3s 65us/step - loss: 1.2347 - acc: 0.5591
Epoch 6/40
50000/50000 [==============================] - 3s 68us/step - loss: 1.1232 - acc: 0.5984
Epoch 7/40
50000/50000 [==============================] - 3s 63us/step - loss: 1.0518 - acc: 0.6264
Epoch 8/40
50000/50000 [==============================] - 3s 64us/step - loss: 0.9759 - acc: 0.6538
Epoch 9/40
50000/50000 [==============================] - 3s 65us/step - loss: 0.9175 - acc: 0.6745
Epoch 10/40
50000/50000 [==============================] - 3s 64us/step - loss: 0.8656 - acc: 0.69

In [12]:
%%time
# Main evaluation loop
y_guess = model.predict(x_test, batch_size=BATCHSIZE)
y_guess = np.argmax(y_guess, axis=-1)
y_truth = np.argmax(y_test, axis=-1)

CPU times: user 258 ms, sys: 100 ms, total: 358 ms
Wall time: 398 ms


In [13]:
print("Accuracy: ", 1.*sum(y_guess == y_truth)/len(y_guess))

Accuracy:  0.8051
